# Environment  

In [1]:
# %%capture --no-stderr
# %pip install --quiet -U langchain langchain_community tiktoken langchain-mistralai scikit-learn langgraph tavily-python bs4

In [2]:
# !pip install scikit-learn

In [1]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

# _set_env("TAVILY_API_KEY")
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

In [2]:
# _set_env("LANGCHAIN_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"
os.environ["LANGCHAIN_PROJECT"] = "estyl"
os.environ['TAVILY_API_KEY'] = "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"
os.environ["OPENAI_API_KEY"] = "sk-svcacct-vZG2VCHIjZUn7fl3EmK51F77F3aPcLnrVf7Bd0-Z0DUzt9Zv9mfomR09b_7gvT3BlbkFJPnOvUAPfOk75znoM8NPQDGS8KAEhSuj82s5I5jmjpvLo7-13URd9cnDBHk_wgA"

## Index

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader

from langchain_core.vectorstores import InMemoryVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
import sys
sys.path.append('chains')
sys.path.append('tools')
sys.path.append('graph')
from chains import *
from tools import *
from graph import *

USER_AGENT environment variable not set, consider setting it to identify your requests.


## LLMs

In [4]:
### Set LLM
from langchain_openai import ChatOpenAI # from langchain_mistralai import ChatMistralAI
mistral_model = "mistral-large-latest" # "open-mixtral-8x22b" 
llm = ChatOpenAI()

loader = PyPDFLoader('pdfs/ESTYL _ STYLE GUIDE (pdf).pdf')

docs = loader.load()

print(len(docs))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = InMemoryVectorStore.from_documents(
    documents=splits, embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

115


In [5]:
import chains

### Router

# Chain the prompt and the structured LLM output
router_chain = get_router_chain(llm)

# Test the router with example questions
question_1 = "How to dress up for a wedding?"
question_2 = "What are the types of agent memory?"

# Invoke the router chain for both questions
result_1 = router_chain.invoke({"question": question_1})
result_2 = router_chain.invoke({"question": question_2})

# Print the results
print(result_1)
print(result_2)

datasource='vectorstore'
datasource='generate'


In [8]:
### Retrieval Grader 

retrieval_grader_chain = get_retrieval_grader_chain(llm)
question = "What is posh style?"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader_chain.invoke({"question": question, "document": doc_txt}))

binary_score='yes'


In [9]:
### Generate

rag_chain = get_generate_chain(llm)

# Run
generation = rag_chain.invoke({"context": docs, "question": "What is posh style?", "messages": []})
print(generation)

ValueError: Invalid input type <class 'langchain_core.messages.ai.AIMessage'>. Must be a PromptValue, str, or list of BaseMessages.

In [16]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import BaseModel, Field
from pydantic import BaseModel, Field

### Answer Grader 

# Data model
class Generation(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(description="Conversation arrived to a point where the user asked a specific outfit advice, 'yes' or 'no'")
    generation: str = Field(description="Generated answer")

def get_structured_llm_grader(llm):
    return llm.with_structured_output(schema=GradeAnswer)

structured_llm_grader = get_structured_llm_grader(llm)

# Prompt 
system = """You are a fashion stylist assistant focused on creating personalized outfit suggestions and providing fashion advice.

If the user engages in small talk, answer appropriately. Then, engage by asking if they are interested in getting a specific outfit suggestion based on the advice, especially if the question is not clear.

If the user talks about topics outside fashion or styling, remind the user that your job is to guide him in styling and provide outfit recommendations.

If the user has a specific request for an outfit or look, confirm if what they are describing is correct. Once confirmed, suggest a complete look including a top, bottom, shoes, and any essential accessories. After suggesting the outfit, ask if it matches what the user had in mind and if they are happy with the choice. Adjust based on their feedback if needed. 

You can suggest outfits based on the context.

Keep the conversation friendly, helpful, and engaging, ensuring that the user feels excited and confident 
about their style choices.\n

Based on the conversation, You will also assess whether the user has made a specific outfit request \n 
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question:\n\n {question} \n\n Context: {context} \n\n Conversation History: {messages}"),
    ]
)

answer_grader_chain = answer_prompt | structured_llm_grader

#     return answer_grader_chain

answer_grader_chain.invoke({"question": question,"generation": "hello"})

GradeAnswer(binary_score='no', explanation="The answer provided does not address the user's question about 'posh style'.")

In [23]:
### Generate

from langchain import hub
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

# Data model
class Generation(BaseModel):
    """Generation and binary score to say whether a complete outfit suggestion is asked by a user."""

    binary_score: str = Field(description="Conversation arrived to a point where the user asked a specific outfit advice, 'yes' or 'no'")
    generation: str = Field(description="Generated answer")
    
def get_structured_generation(llm):
    return llm.with_structured_output(schema=Generation)

# Prompt
prompt = hub.pull("rlm/rag-prompt")
# prompt.messages[0].prompt.template
GENERATION_INSTRUCTIONS = """
You are a fashion stylist assistant focused on creating personalized outfit suggestions and providing fashion advice.

If the user engages in small talk, answer appropriately. Then, engage by asking if they are interested in getting a specific outfit suggestion based on the advice, especially if the question is not clear.

If the user talks about topics outside fashion or styling, remind the user that your job is to guide him in styling and provide outfit recommendations.

If the user has a specific request for an outfit or look, confirm if what they are describing is correct. Once confirmed, suggest a complete look including a top, bottom, shoes, and any essential accessories. After suggesting the outfit, ask if it matches what the user had in mind and if they are happy with the choice. Adjust based on their feedback if needed. 

You can suggest outfits based on the context.

Keep the conversation friendly, helpful, and engaging, ensuring that the user feels excited and confident 
about their style choices.\n

Based on the conversation, You will also assess whether the user has made a specific outfit request \n 
Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question.

Output in JSON format:

Generation: "your response"
Binary score: "outfit requested"

"""

# LLM
# llm = ChatMistralAI(model=mistral_model, temperature=0)
# memory = MemorySaver()
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# LLM with structured output
structured_generation = get_structured_generation(llm)

# Define the prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", GENERATION_INSTRUCTIONS),
        ("human", "User question:\n\n {question} \n\n Context: {context} \n\n Conversation History: {messages}"),
    ]
)

# Chain
rag_chain = prompt | structured_generation #StrOutputParser()

# return rag_chain

In [24]:
# ### Generate

# rag_chain = get_generate_chain(llm)

# Run
generation = rag_chain.invoke({"context": docs, "question": "What is posh style?", "messages": []})
print(generation)

binary_score='no' generation='Posh style is all about sophistication, elegance, and luxury. It involves attention to detail, quality fabrics, and neutral color palettes. Classic, well-tailored dresses in luxurious fabrics are a staple of posh style. Do you want a specific outfit suggestion based on this posh style description?'


In [ ]:
### Hallucination Grader 

# Data model
hallucination_grader_chain = get_hallucination_grader_chain(llm)
hallucination_grader_chain.invoke({"documents": docs, "generation": generation})

In [ ]:
### Answer Grader 

answer_grader_chain = get_answer_grader_chain(llm)
answer_grader_chain.invoke({"question": question,"generation": generation})

## Web Search Tool

In [ ]:
web_search_tool = get_web_search_tool()
web_search_tool.invoke({"query": question})

In [ ]:
web_search_tool

# Graph 

### Graph state

## Build Graph

In [ ]:
chat_graph = create_graph()
display(Image(chat_graph.get_graph().draw_mermaid_png()))

Task exception was never retrieved
future: <Task finished name='Task-7' coro=<AsyncClient.aclose() done, defined at /Users/mokkastudios/opt/anaconda3/envs/estyl/lib/python3.11/site-packages/httpx/_client.py:2024> exception=ImportError("cannot import name 'ExceptionGroup' from 'anyio._core._exceptions' (/Users/mokkastudios/opt/anaconda3/envs/estyl/lib/python3.11/site-packages/anyio/_core/_exceptions.py)")>
Traceback (most recent call last):
  File "/Users/mokkastudios/opt/anaconda3/envs/estyl/lib/python3.11/site-packages/anyio/_core/_eventloop.py", line 162, in get_async_backend
KeyError: 'asyncio'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/mokkastudios/opt/anaconda3/envs/estyl/lib/python3.11/site-packages/httpx/_client.py", line 2031, in aclose
    await self._transport.aclose()
  File "/Users/mokkastudios/opt/anaconda3/envs/estyl/lib/python3.11/site-packages/httpx/_transports/default.py", line 389, in aclose


In [29]:
from pprint import pprint
inputs = {"question": "What is posh style"}
config = {"configurable": {"thread_id": "1"}}
for output in chat_graph.stream(inputs, config):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
value["generation"]

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
documents 2:  [Document(id='f5eabe7f-9616-4b2b-b18a-2c7d3b97f6d0', metadata={'source': 'pdfs/ESTYL _ STYLE GUIDE (pdf).pdf', 'page': 2}, page_content="POSHPosh Style typically refers to a sophisticated, elegant, and polished fashion aesthetic associated with luxury and class. It's often aligned with upper-class sensibilities and chic, timeless design elements. The posh look is characterized by attention to detail, quality fabrics, a neutral or muted color palette, and an understated sense of opulence.   Classic, well-tailored dresses are the hallmark of posh style. Think elegant sheath dresses, midi or tea-length dresses, and A-line silhouettes in luxurious fabrics like silk, satin, or cashmere. Timeless designs in neutral shades like black, white, navy, or cream dominate, though bold jewel tones (emerald, burgundy, royal blue) are also seen. Minimalist detailing such as clean lines, structured silhouettes, and subtle embe

"Posh style refers to a sophisticated, elegant, and polished fashion aesthetic associated with luxury and class. It's characterized by attention to detail, quality fabrics, and neutral or muted color palettes. Posh style includes classic, well-tailored dresses, structured blazers, and polished leather accessories."

In [38]:
from pprint import pprint
inputs = {"question": "What is posh style"}
config = {"configurable": {"thread_id": "1"}}
events = chat_graph.stream(inputs, config)
for output in events:
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
value["generation"]

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
---RETRIEVE---
documents 2:  [Document(id='f5eabe7f-9616-4b2b-b18a-2c7d3b97f6d0', metadata={'source': 'pdfs/ESTYL _ STYLE GUIDE (pdf).pdf', 'page': 2}, page_content="POSHPosh Style typically refers to a sophisticated, elegant, and polished fashion aesthetic associated with luxury and class. It's often aligned with upper-class sensibilities and chic, timeless design elements. The posh look is characterized by attention to detail, quality fabrics, a neutral or muted color palette, and an understated sense of opulence.   Classic, well-tailored dresses are the hallmark of posh style. Think elegant sheath dresses, midi or tea-length dresses, and A-line silhouettes in luxurious fabrics like silk, satin, or cashmere. Timeless designs in neutral shades like black, white, navy, or cream dominate, though bold jewel tones (emerald, burgundy, royal blue) are also seen. Minimalist detailing such as clean lines, structured silhouettes, and subtle embe

'Posh style refers to a sophisticated, elegant, and polished fashion aesthetic associated with luxury and class, often aligned with upper-class sensibilities. It involves attention to detail, quality fabrics, a neutral or muted color palette, and an understated sense of opulence. Classic, well-tailored dresses and structured blazers are key elements of posh style.'

In [39]:
events

<generator object Pregel.stream at 0x11fe8fde0>

In [30]:
value["generation"]

"Posh style refers to a sophisticated, elegant, and polished fashion aesthetic associated with luxury and class. It's characterized by attention to detail, quality fabrics, and neutral or muted color palettes. Posh style includes classic, well-tailored dresses, structured blazers, and polished leather accessories."

In [ ]:
inputs = {
    "question": "hello",
    "llm": llm,  # Pass LLM # Pass retriever
}

config = {"configurable": {"thread_id": "1"}}

# Run the graph with the initialized state
for output in chat_graph.stream(inputs, config):
    print(output)

In [16]:
dir(chat_graph)

['InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abatch_with_config',
 '_abc_impl',
 '_acall_with_config',
 '_aprepare_state_snapshot',
 '_atransform_stream_with_config',
 '_batch_with_config',
 '_call_with_config',
 '_defaults',
 '_is_protocol',
 '_prepare_state_snapshot',
 '_transform_stream_with_config',
 'abatch',
 'abatch_as_completed',
 'aget_state',
 'aget_state_history',
 'aget_subgraphs',
 'ainvoke',
 'as_tool',
 'assign',
 'astream',
 'astream_events',
 'astream_log',
 'atra

In [19]:
dir(workflow)

['__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_schema',
 '_all_edges',
 'add_conditional_edges',
 'add_edge',
 'add_node',
 'branches',
 'channels',
 'compile',
 'compiled',
 'config_schema',
 'edges',
 'input',
 'managed',
 'nodes',
 'output',
 'schema',
 'schemas',
 'set_conditional_entry_point',
 'set_entry_point',
 'set_finish_point',
 'support_multiple_edges',
 'validate',
 'waiting_edges']

In [14]:
chat_graph.get_graph

<bound method CompiledGraph.get_graph of <langgraph.graph.state.CompiledStateGraph object at 0x108fa2350>>

In [28]:
type(workflow)

langgraph.graph.state.StateGraph

In [29]:
from langgraph.graph import StateGraph

In [30]:
StateGraph??

In [33]:
dir(chat_graph)

['InputType',
 'OutputType',
 '__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__or__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__ror__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abatch_with_config',
 '_abc_impl',
 '_acall_with_config',
 '_aprepare_state_snapshot',
 '_atransform_stream_with_config',
 '_batch_with_config',
 '_call_with_config',
 '_defaults',
 '_is_protocol',
 '_prepare_state_snapshot',
 '_transform_stream_with_config',
 'abatch',
 'abatch_as_completed',
 'aget_state',
 'aget_state_history',
 'aget_subgraphs',
 'ainvoke',
 'as_tool',
 'assign',
 'astream',
 'astream_events',
 'astream_log',
 'atra

In [39]:
chat_graph.channels["documents"]

AttributeError: 'LastValue' object has no attribute 'value'